In [1]:
import pandas as pd
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
import numpy

In [2]:
train = pd.read_csv('train.csv', sep = '\t', index_col='id') #memory error

ParserError: Error tokenizing data. C error: out of memory

In [3]:
# had to do this way, because of memory error
def get_data(file):
    mylist = []
    for chunk in  pd.read_csv(file, sep='\t', chunksize=20000):
        mylist.append(chunk)
    dataframe = pd.concat(mylist, axis= 0)
    dataframe.set_index('id', inplace = True)
    del mylist
    return dataframe
    

In [4]:
train = get_data('train.csv')

In [5]:
test = get_data('test.csv')
#other = get_data('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [6]:
y = train.target

In [7]:
train =train.drop('target', axis = 1)

In [ ]:
train.head()

In [38]:
def preproc_data(dataframe):
    df_output = dataframe
    df_output = df_output.drop('description', axis = 1)
    df_output['name'] = df_output['name'].str.lower()
    df_output['name'] = df_output['name'].str.replace('(', '').replace(')', '').replace('/', '')
    df_output['name'] = df_output['name'].str.split(r" \b([\w+-]+)\b g" )
    df_output['name'] = [" ".join(name) for name in df_output['name'].values]
    return df_output

In [39]:
train_work = preproc_data(train)

In [40]:
test_work = preproc_data(test)

In [41]:
test_work.head()

,name
id,
200000,дизайнер-консультант мебели
200001,продавец-консультант тц на пушкина)
200002,менеджер по продажам
200003,продавец-консультант в магазин одежды тц волна...
200004,специалист по охране труда


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [35]:
t = train_work.append(test_work)['name']

In [54]:
cv = CountVectorizer()
cv.fit_transform(train_work.append(test_work)['name'])

<370179x30202 sparse matrix of type '<class 'numpy.int64'>'
	with 1423425 stored elements in Compressed Sparse Row format>

In [49]:
cv.vocabulary_

{'заведующий': 44089,
 'отделом': 88458,
 'секцией': 120697,
 'магазин': 65977,
 'york': 22162,
 'уручье': 140599,
 'заведующий отделом': 44139,
 'отделом секцией': 88507,
 'секцией магазин': 120701,
 'магазин york': 66153,
 'york уручье': 22165,
 'наладчик': 79249,
 'станков': 127404,
 'манипуляторов': 68986,
 'пу': 110403,
 'наладчик станков': 79319,
 'станков манипуляторов': 127406,
 'манипуляторов пу': 68988,
 'разработчик': 112589,
 'криптограф': 62571,
 'разработчик криптограф': 113030,
 'фрезеровщик': 143116,
 'мерчендайзер': 73622,
 'продавец': 101151,
 'консультант': 57951,
 'мерчендайзер продавец': 74125,
 'продавец консультант': 101520,
 'мастер': 70126,
 'по': 92478,
 'эксплуатации': 147627,
 'зданий': 46065,
 'мастер по': 70299,
 'по эксплуатации': 94677,
 'эксплуатации зданий': 147673,
 'торговый': 134540,
 'представитель': 97959,
 'торговый представитель': 134562,
 'менеджер': 72058,
 'продажам': 103273,
 'менеджер по': 72804,
 'по продажам': 93943,
 'работе': 110775,
 '

In [55]:
train_work_transform = cv.transform(train_work.name)
test_work_transform = cv.transform(test_work.name)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(train_work_transform, y, test_size=0.33, random_state=42)
lr = LogisticRegression()

lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
print(roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]))


0.9837849844331357


In [58]:
predict = lr.predict_proba(test_work_transform)
result_predict = pd.DataFrame(data={'id': test.index, 'target': predict[:,1]})
result_predict.to_csv('hw_06_submit_1.csv', index=False)